In [1]:
import requests
import os
import json
import pandas as pd
import numpy as np


In [4]:
work_path=r'D:\university\研究生\tourism_forecasting_competition\submit\code and supplementary materials' ##work_path must be adjusted before running the code.
os.chdir(work_path)

In [5]:
keywords_dict=np.load('keywords.npy',allow_pickle=True).item()  ##load the keywords dict

In [6]:
keywords_dict

{'加拿大': Index(['加拿大旅游', '加拿大酒店', '加拿大签证', '加拿大航空', '温哥华', '蒙特利尔', '渥太华'], dtype='object'),
 '智利': Index(['智利旅游', '智利签证', '圣地亚哥'], dtype='object'),
 '墨西哥': Index(['墨西哥旅游', '墨西哥签证'], dtype='object'),
 '台湾': Index(['台湾旅游', '台湾旅游攻略', '台湾酒店', '台湾签证', '台湾美食', '台湾小吃', '台湾购物'], dtype='object'),
 '香港': Index(['香港旅游', '香港旅游攻略', '香港航空', '香港酒店', '香港签证', '香港美食', '香港小吃', '香港购物',
        '香港旅游地图', '香港旅游景点', '港澳通行证'],
       dtype='object'),
 '日本': Index(['日本旅游', '日本旅游攻略', '日本航空', '日本酒店', '日本签证', '日本美食', '日本小吃', '日本购物',
        '日本旅游地图', '日本旅游景点', '东京', '大阪', '名古屋'],
       dtype='object'),
 '韩国': Index(['韩国旅游', '韩国旅游攻略', '韩国酒店', '韩国签证', '韩国美食', '韩国购物', '韩国旅游景点', '首尔', '济州岛',
        '釜山'],
       dtype='object'),
 '澳门': Index(['澳门旅游', '澳门旅游攻略', '澳门航空', '澳门酒店', '澳门签证', '澳门美食', '港澳通行证'], dtype='object'),
 '马尔代夫': Index(['马尔代夫旅游', '马尔代夫旅游攻略', '马尔代夫酒店', '马尔代夫签证'], dtype='object'),
 '柬埔寨': Index(['柬埔寨旅游', '柬埔寨签证', '吴哥窟'], dtype='object'),
 '印尼': Index(['印尼旅游', '印尼签证', '巴厘岛'], dtype='object'),
 '新加坡': Inde

In [ ]:
word_url = 'http://index.baidu.com/api/SearchApi/thumbnail?area=0&word={}'
 
def get_html(url):
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/113.0.0.0 Safari/537.36 Edg/113.0.1774.35",
        "Host": "index.baidu.com",
        "Referer": "http://index.baidu.com/v2/main/index.html",
        "Cipher-Text": "1683640803370_1683703585214_jOeRpyxsdtGlDTGrBbZqj35wdrlA3C3hUDapZa0rB8O0H0aY7YADSkikLuIOT3vouLE64ezNLOAjqEjKho7vVDu5kW/1MKJvFfIRWQT1Pi+Rm7RoG9DIq7kLc9Tkq27ib6KvcrbM1bADI67Rrlae7Xkqu8s208JTx+QgL6BagojiA7Ks9Jx9Tsv4b8hOvY8Yi/IqUDoOlgf/ViDiFl1hE+jsN8Efpt4FxRVT8i1UwgFyIBgC6DNNgxLo0T7GkUUd21sjBjIA7NRBGUUvKglOqCrHsPVLZi+DMBLmIFnnmBTsBhLcQwc8bMvL/foSJpn9AaPjcHaL9d1p8/3GE62a2aNybc0mIQRSmjoAgLfuCxNYjIxOoyuupscsNAF3DBX/ps/jWZgYLb3iGO+nV6br4x9CckpLu2xy5R2t1Igqz7DBDuLJH+i7WzJdf6+LKkDkcEYYIuWUTzMhVEUms+DKiA=="
    }
    cookies = {
        'Cookie': '__bid_n=187556eeb9d9f14ef04207; BAIDU_WISE_UID=wapp_1680766238391_861; BIDUPSID=8BF8887FE98B8F3095EDEFE3F9A5C228; PSTM=1680768667; BAIDUID=6E8618ADC330286A422B9E7F7D5B9E54:FG=1; BAIDUID_BFESS=6E8618ADC330286A422B9E7F7D5B9E54:FG=1; BDUSS=URsUm1pRjU5NDVXVThPRkg0cm5LNERCOFp2NHd3UjFFbXBHRnNreUoxR1JHbFprRVFBQUFBJCQAAAAAAAAAAAEAAABIl1qZcXVpZXRyYWJiaXRhbmQAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAJGNLmSRjS5ka; ZFY=2csmpPSpawU9u2i7GAT9ltDPfH41JRzrXi8Ukr1mkhk:C; Hm_lvt_d101ea4d2a5c67dab98251f0b5de24dc=1683341849,1683527825; BCLID=12113974424415830444; BDSFRCVID=e-4OJeC62r80bf3fSS9vrOgFmeKMMy6TH6_nHeHPgh9WAWrTqaLNEG0PfM8g0Ku-OBJCogKK5mOTH6KF_2uxOjjg8UtVJeC6EG0Ptf8g0M5; H_BDCLCKID_SF=JJuH_DP5f-OqDjrn5b5EKKCShUFsKCuJB2Q-5KL--DP2HRLRblb6eJKXXUJl24QiBmj8_fbdJf7_Ob5S0-Jseq-F3p5nb4orBeTxoUJDBCnJhhvGqfnIyxtebPRit4r9Qg-qahQ7tt5W8ncFbT7l5hKpbt-q0x-jLTnhVn0MBCK0HPonHjL5jjcB3J; BCLID_BFESS=12113974424415830444; BDSFRCVID_BFESS=e-4OJeC62r80bf3fSS9vrOgFmeKMMy6TH6_nHeHPgh9WAWrTqaLNEG0PfM8g0Ku-OBJCogKK5mOTH6KF_2uxOjjg8UtVJeC6EG0Ptf8g0M5; H_BDCLCKID_SF_BFESS=JJuH_DP5f-OqDjrn5b5EKKCShUFsKCuJB2Q-5KL--DP2HRLRblb6eJKXXUJl24QiBmj8_fbdJf7_Ob5S0-Jseq-F3p5nb4orBeTxoUJDBCnJhhvGqfnIyxtebPRit4r9Qg-qahQ7tt5W8ncFbT7l5hKpbt-q0x-jLTnhVn0MBCK0HPonHjL5jjcB3J; FPTOKEN=aZvEykJ8inJdiK8J+26TQ2sRBdGBKxH3MC/954ErMFhu8HC7s9ukuidLFykCy+hEK9CoGy7cSl6AAqS8Zaojvs6OCOc5hSdzpbJJq6mLUqcSRJQ3hZdAN7h9QF5eM/igYhO+4jJyBpK48anIA32b2q8n9Gy941154aL0aQzUC83zSeBWAiNl3eIvDMopoD5Xdku/KPo124O7lVFF6kRiTPDd07XGum8mACedo5OaVWESEiZJFCdIoAMQkVKZhLp4CUCSciRIGAheu0KFDga1eNFfw5YqKicAL16h6x1HqHJAWpkxawQ+ldHKB58/DoDoL+WP1itzNy2FMajrtRvWgiBRHMn2PmgIxcgbCTT37WyN6cGZRK/7HCcsRTu281tUa3K8YkT6UpUDQQHPk37hrw==|RXxAOFTNHH9FNJvD1IWAjp0Nn78jcMB9m9iZ6mFkNo4=|10|6e7d938d6db0338f219c19f21b32919d; SIGNIN_UC=70a2711cf1d3d9b1a82d2f87d633bd8a043369748338d1kGD7uw8OAwomMLq8s8JWIsEPNz151j4nrTu40YH7Xn98cpT4BG1TdSH7F4In+2qHxdlJZJpKh2q3qlZO93oq4CVlwtvKx8LNS4gjDhVLSJZ5djGXzommZn6rgAuWcsOs9eLweFFAbToVBI+mhFP1ki+Xh16eaIgZwkQvwq6ATUjMVMdvz6KAc++2mfuRqoq9E/YvTRZFa2wQpa4UiIwRhHIKFvgu73S9MbZYzILN4Brndl/ZUx8Cvhahhyv9FCtHYfqJ1Me392vBM0wGg0pe1+D9mNubgiXOFcpUHanc=28619630724356211238138017686683; __cas__rn__=433697483; __cas__st__212=509837eea2a54c8ccb6d39711cd5c2bc7923ec2384d576f3a6eadfd67014d821f8da69903e138f4275225e05; __cas__id__212=47790421; CPID_212=47790421; CPTK_212=1015076435; bdindexid=ng9fk4fiumf586222g88qauof6; H_PS_PSSID=38516_36543_38529_38470_38468_38594_38595_36807_38486_26350_38568_22160_38543; BA_HECTOR=218l002g2521a1840kal2gdk1i5mggd1n; ab_sr=1.0.1_M2RkNmUwMmMxZTE3MWY0ZmYwNzM0Njc3MmQ4MzZjZDgyNzE5MGY3ZWQ3YzRlODhlNTEzZDNkMDMyZWI2M2VlNjY4MWRiMmE4NmVjY2EwZTJhYmY2OGQzMTYxOTI2NDBhZTUwOWQ0YTQ0ZGY0OWE0YzlhMjkxZWY5ZmY2ZDMxOWMzNDA3ZTA3YTk0ZThkZDFjMDhkYmFiODY0OGM2MzI3Ng==; RT="z=1&dm=baidu.com&si=32438c12-18de-46ef-a8b1-29ac2eac21e1&ss=lhhadex8&sl=g&tt=cho&bcn=https://fclog.baidu.com/log/weirwood?type=perf"; Hm_lpvt_d101ea4d2a5c67dab98251f0b5de24dc=1683703585; BDUSS_BFESS=URsUm1pRjU5NDVXVThPRkg0cm5LNERCOFp2NHd3UjFFbXBHRnNreUoxR1JHbFprRVFBQUFBJCQAAAAAAAAAAAEAAABIl1qZcXVpZXRyYWJiaXRhbmQAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAJGNLmSRjS5ka'
    }
    response = requests.get(url, headers=headers, cookies=cookies)
    return response.text
 
 
def decrypt(t, e):
    n = list(t)
    i = list(e)
    a = {}
    result = []
    ln = int(len(n) / 2)
    start = n[ln:]
    end = n[:ln]
    for j, k in zip(start, end):
        a.update({k: j})
    for j in e:
        result.append(a.get(j))
    return ''.join(result)
 
 
def get_ptbk(uniqid):
    url = 'http://index.baidu.com/Interface/ptbk?uniqid={}'
    resp = get_html(url.format(uniqid))
    return json.loads(resp)['data']
 
 
def get_data(keyword, start='2011-01-01', end='2023-05-31'):
    url = "https://index.baidu.com/api/SearchApi/index?area=0&word=[[%7B%22name%22:%22{}%22,%22wordType%22:1%7D]]&startDate={}&endDate={}".format(keyword, start, end)
    data = get_html(url)
    data = json.loads(data)
    uniqid = data['data']['uniqid']
    data = data['data']['userIndexes'][0]['all']['data']
    ptbk = get_ptbk(uniqid)
    result = decrypt(ptbk, data)
    result = result.split(',')
    return result
 

In [ ]:
###retrieve the search data 
###the Chinese translation of country name is used for convenience in the following codes.
country_name=["加拿大",'智利','墨西哥','台湾','香港','日本','韩国','澳门','马尔代夫','柬埔寨','印尼','新加坡','新西兰','美国','泰国','土耳其','澳大利亚','夏威夷','奥地利','捷克']
for country in country_name:
    df=pd.DataFrame()
    df['date']=pd.date_range(start="2010-12-27", freq="7D", periods=649)
    start_date = "2011-01-01"
    end_date = "2023-05-31"
    keywords=keywords_dict[country]
    for keyword in keywords:
        temp=get_data(keyword, start_date, end_date)
        df[keyword]=temp
    df.to_excel(work_path+'\\'+'baidu_index\{}.xlsx'.format(country),encoding='utf-8')